<a href="https://colab.research.google.com/github/SheidaEmdadi/MRI-Project/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip  install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu121.html


In [2]:
! pip install  torch_geometric

In [3]:
! pip install --upgrade pygod

In [4]:
import torch
print(torch.__version__)


2.2.1+cu121


In [5]:
import numpy as np
import scipy.sparse as sp
import time
import scipy
import random
import pdb
import copy
import os
import sys
import argparse
import pandas as pd

import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GCNConv,GINConv,SAGEConv,GATConv,PNAConv, GraphSAGE
from torch_geometric.utils import to_dense_adj
from pygod.utils import load_data

from sklearn.metrics import  roc_auc_score


 This *eval_roc_auc* function is borrowed from: https://github.com/pygod-team/pygod/blob/main/pygod/metric/metric.py

 Author: Yingtong Dou <ytongdou@gmail.com>, Kay Liu <zliu234@uic.edu>

 License: BSD 2 clause

In [6]:
def eval_roc_auc(label, score):
    roc_auc = roc_auc_score(y_true=label, y_score=score)
    return roc_auc


#Helper Function

In [7]:
def aug_random_edge(input_adj, perturb_percent=0.2, drop_edge = True, add_edge = True, self_loop = True):

    aug_adj = copy.deepcopy(input_adj)
    nb_nodes = input_adj.shape[0]
    edge_index = (input_adj>0).nonzero().t()

    edge_dict = {}
    for i in range(nb_nodes):
        edge_dict[i] = set()

    for edge in edge_index:
        i,j = edge[0],edge[1]
        i = i.item()
        j = j.item()
        edge_dict[i].add(j)
        edge_dict[j].add(i)

    if drop_edge:
        for i in range(nb_nodes):
            d = len(edge_dict[i])
            node_list = list(edge_dict[i])
            num_edge_to_drop = int(d * perturb_percent)

            sampled_nodes = random.sample(node_list, num_edge_to_drop)

            for j in sampled_nodes:
                aug_adj[i][j] = 0
                aug_adj[j][i] = 0


    node_list = [i for i in range(nb_nodes)]

    add_list = []
    for i in range(nb_nodes):
        d = len(edge_dict[i])
        num_edge_to_add = int(d * perturb_percent)
        sampled_nodes = random.sample(node_list, num_edge_to_add)
        for j in sampled_nodes:
            add_list.append((i,j))

    if add_edge:
        for i in add_list:
            aug_adj[i[0]][i[1]] = 1
            aug_adj[i[1]][i[0]] = 1

    if self_loop:
        for i in range(nb_nodes):
            aug_adj[i][i] = 1
            aug_adj[i][i] = 1


    return aug_adj


def _aug_random_edge(nb_nodes, edge_index, perturb_percent=0.2, drop_edge = True, add_edge = True, self_loop = True, use_avg_deg = True):


    total_edges = edge_index.shape[1]
    avg_degree = int(total_edges/nb_nodes)


    edge_dict = {}
    for i in range(nb_nodes):
        edge_dict[i] = set()

    for edge in edge_index:
        i,j = edge[0],edge[1]
        i = i.item()
        j = j.item()
        edge_dict[i].add(j)
        edge_dict[j].add(i)

    if drop_edge:
        for i in range(nb_nodes):

            d = len(edge_dict[i])
            if use_avg_deg:
                num_edge_to_drop = avg_degree
            else:
                num_edge_to_drop = int(d * perturb_percent)

            node_list = list(edge_dict[i])
            num_edge_to_drop = min(num_edge_to_drop, d)
            sampled_nodes = random.sample(node_list, num_edge_to_drop)

            for j in sampled_nodes:
                edge_dict[i].discard(j)
                edge_dict[j].discard(i)

    node_list = [i for i in range(nb_nodes)]

    add_list = []
    for i in range(nb_nodes):

        if use_avg_deg:
            num_edge_to_add =  avg_degree
        else:
            d = len(edge_dict[i])
            num_edge_to_add = int(d * perturb_percent)

        sampled_nodes = random.sample(node_list, num_edge_to_add)
        for j in sampled_nodes:
            add_list.append((i,j))

    if add_edge:
        for edge in add_list:
            u = edge[0]
            v = edge[1]

            edge_dict[u].add(v)
            edge_dict[v].add(u)

    if self_loop:
        for i in range(nb_nodes):
            edge_dict[i].add(i)

    updated_edges = set()
    for i in range(nb_nodes):
        for j in edge_dict[i]:
            updated_edges.add((i,j))
            updated_edges.add((j,i))

    row = []
    col = []
    for edge in updated_edges:
        u = edge[0]
        v = edge[1]
        row.append(u)
        col.append(v)

    aug_edge_index = [row,col]
    aug_edge_index = torch.tensor(aug_edge_index)

    return aug_edge_index


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    features = features.squeeze()
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features
    # return features, sparse_to_tuple(features)

def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

#MLP

In [8]:
class MLP(nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dim, output_dim):

        super(MLP, self).__init__()

        self.linear_or_not = True  # default is linear model
        self.num_layers = num_layers

        if num_layers < 1:
            raise ValueError("number of layers should be positive!")
        elif num_layers == 1:
            # Linear model
            self.linear = nn.Linear(input_dim, output_dim)
        else:
            # Multi-layer model
            self.linear_or_not = False
            self.linears = torch.nn.ModuleList()
            self.batch_norms = torch.nn.ModuleList()

            self.linears.append(nn.Linear(input_dim, hidden_dim))
            for layer in range(num_layers - 2):
                self.linears.append(nn.Linear(hidden_dim, hidden_dim))
            self.linears.append(nn.Linear(hidden_dim, output_dim))

            for layer in range(num_layers - 1):
                self.batch_norms.append(nn.BatchNorm1d((hidden_dim)))

    def forward(self, x):
        if self.linear_or_not:
            # If linear model
            return self.linear(x)
        else:
            # If MLP
            h = x
            for layer in range(self.num_layers - 1):
                h = self.linears[layer](h)

                if len(h.shape) > 2:
                    h = torch.transpose(h, 0, 1)
                    h = torch.transpose(h, 1, 2)

                # h = self.batch_norms[layer](h)

                if len(h.shape) > 2:
                    h = torch.transpose(h, 1, 2)
                    h = torch.transpose(h, 0, 1)

                h = F.relu(h)
                # h = F.relu(self.linears[layer](h))

            return self.linears[self.num_layers - 1](h)

#GNN Model

In [19]:
class GNN(nn.Module):
    def __init__(self, in_dim, out_dim):

        super(GNN, self).__init__()

        self.mlp0 = MLP(3, in_dim, out_dim, out_dim)

        self.graphconv1 = GCNConv(out_dim, out_dim, aggr='mean')


        self.mlp1 = nn.Linear(out_dim,1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, x, edge_index):
        h0 = self.mlp0(x)
        h1 = self.graphconv1(h0,edge_index)
        h2 = self.mlp1(h1)
        h2 = self.relu(h2)
        p = torch.exp(h2)
        return p



#Parsers

In [10]:
weibo_parser = argparse.ArgumentParser("GAD-EBM: Graph Anomaly Detection via Energy-based Model")

weibo_parser.add_argument('-f')
weibo_parser.add_argument('--dataset',          type=str,           default="weibo")
weibo_parser.add_argument('--perturb_percent',  type=float,         default=0.05)
weibo_parser.add_argument('--seed',             type=int,           default=42)
weibo_parser.add_argument('--nb_epochs',        type=int,           default=200)
weibo_parser.add_argument('--hidden_dim',       type=int,           default=64)
weibo_parser.add_argument('--lr',               type=float,         default=0.1)
weibo_parser.add_argument('--l2_coef',          type=float,         default=100)
weibo_parser.add_argument('--gpu',              type=int,           default=0)
weibo_parser.add_argument('--save_name',        type=str,           default='try.pkl')
weibo_parser.add_argument('--drop_edge',        type=bool,          default=True)
weibo_parser.add_argument('--add_edge',         type=bool,          default=False)
weibo_parser.add_argument('--self_loop',        type=bool,          default=False)
weibo_parser.add_argument('--use_avg_deg',      type=bool,          default=False)
weibo_parser.add_argument('--preprocess_feat',  type=bool,          default=False)
weibo_parser.add_argument('--num_neigh',        type=int,           default=1)

_StoreAction(option_strings=['--num_neigh'], dest='num_neigh', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [17]:
disney_parser = argparse.ArgumentParser("GAD-EBM: Graph Anomaly Detection via Energy-based Model")

disney_parser.add_argument('-f')
disney_parser.add_argument('--dataset',          type=str,           default="disney")
disney_parser.add_argument('--perturb_percent',  type=float,         default=0.05)
disney_parser.add_argument('--seed',             type=int,           default=42)
disney_parser.add_argument('--nb_epochs',        type=int,           default=200)
disney_parser.add_argument('--hidden_dim',       type=int,           default=16)
disney_parser.add_argument('--lr',               type=float,         default=0.01)
disney_parser.add_argument('--l2_coef',          type=float,         default=0)
disney_parser.add_argument('--gpu',              type=int,           default=0)
disney_parser.add_argument('--save_name',        type=str,           default='try.pkl')
disney_parser.add_argument('--drop_edge',        type=bool,          default=True)
disney_parser.add_argument('--add_edge',         type=bool,          default=False)
disney_parser.add_argument('--self_loop',        type=bool,          default=True)
disney_parser.add_argument('--preprocess_feat',  type=bool,          default=True)
disney_parser.add_argument('--use_avg_deg',      type=bool,          default=False)
disney_parser.add_argument('--num_neigh',        type=int,           default=1)

_StoreAction(option_strings=['--num_neigh'], dest='num_neigh', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [12]:
reddit_parser = argparse.ArgumentParser("GAD-EBM: Graph Anomaly Detection via Energy-based Model")

reddit_parser.add_argument('-f')
reddit_parser.add_argument('--dataset',          type=str,           default="reddit")
reddit_parser.add_argument('--perturb_percent',  type=float,         default=0.05)
reddit_parser.add_argument('--seed',             type=int,           default=10)
reddit_parser.add_argument('--nb_epochs',        type=int,           default=100)
reddit_parser.add_argument('--hidden_dim',       type=int,           default=16)
reddit_parser.add_argument('--lr',               type=float,         default=0.001)
reddit_parser.add_argument('--l2_coef',          type=float,         default=10.0)
reddit_parser.add_argument('--gpu',              type=int,           default=0)
reddit_parser.add_argument('--save_name',        type=str,           default='try.pkl')
reddit_parser.add_argument('--drop_edge',        type=bool,          default=True)
reddit_parser.add_argument('--add_edge',         type=bool,          default=False)
reddit_parser.add_argument('--self_loop',        type=bool,          default=True)
reddit_parser.add_argument('--preprocess_feat',  type=bool,          default=True)
reddit_parser.add_argument('--use_avg_deg',      type=bool,          default=False)
reddit_parser.add_argument('--num_neigh',        type=int,           default=1)

_StoreAction(option_strings=['--num_neigh'], dest='num_neigh', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [13]:
books_parser = argparse.ArgumentParser("GAD-EBM: Graph Anomaly Detection via Energy-based Model")

books_parser.add_argument('-f')
books_parser.add_argument('--dataset',          type=str,           default="books")
books_parser.add_argument('--perturb_percent',  type=float,         default=0.05)
books_parser.add_argument('--seed',             type=int,           default=10)
books_parser.add_argument('--nb_epochs',        type=int,           default=100)
books_parser.add_argument('--hidden_dim',       type=int,           default=16)
books_parser.add_argument('--lr',               type=float,         default=0.01)
books_parser.add_argument('--l2_coef',          type=float,         default=10)
books_parser.add_argument('--gpu',              type=int,           default=0)
books_parser.add_argument('--save_name',        type=str,           default='try.pkl')
books_parser.add_argument('--drop_edge',        type=bool,          default=True)
books_parser.add_argument('--add_edge',         type=bool,          default=False)
books_parser.add_argument('--self_loop',        type=bool,          default=True)
books_parser.add_argument('--preprocess_feat',  type=bool,          default=True)
books_parser.add_argument('--use_avg_deg',      type=bool,          default=False)
books_parser.add_argument('--num_neigh',        type=int,           default=1)


_StoreAction(option_strings=['--num_neigh'], dest='num_neigh', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

#Main

In [20]:
print("please type down your desired dataset and press enter: \n Choose one among this list: \n weibo, disney, reddit, books")
input_p_name = input()
print("your dataset "+ input_p_name + " is being processed.")


if input_p_name == "weibo" :
  args = weibo_parser.parse_args()
elif input_p_name == "disney" :
  args = disney_parser.parse_args()
elif input_p_name == "reddit" :
  args = reddit_parser.parse_args()
elif input_p_name == "books" :
  args = books_parser.parse_args()


print('-' * 400)
print(args)
print('-' * 400)

dataset_str = args.dataset
perturb_percent = args.perturb_percent

seed = args.seed
if input_p_name == "disney":
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

nb_epochs = args.nb_epochs
lr = args.lr
l2_coef = args.l2_coef
hidden_dim = args.hidden_dim
k = args.num_neigh



data = load_data(dataset_str)
edge_index = data.edge_index


adj = to_dense_adj(edge_index).squeeze()
features = data.x
labels = data.y
y = labels.bool()

anomaly_nodes = np.nonzero(y)

nb_nodes = features.shape[0]  # total node
input_dim = features.shape[1]   # total features



model = GNN(input_dim, hidden_dim)
optimiser = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_coef)


if args.preprocess_feat:
    features = preprocess_features(features)




st = time.time()
mx_auc = 0

aug_edge_indexes = []

for i in range(k):
    aug_edge_index = _aug_random_edge(nb_nodes,edge_index, perturb_percent=perturb_percent,drop_edge=args.drop_edge,
                                  add_edge=args.add_edge,self_loop=args.self_loop, use_avg_deg = args.use_avg_deg)


    aug_edge_indexes.append(aug_edge_index)



features = torch.FloatTensor(features[np.newaxis])


losses = []

for epoch in range(nb_epochs):

    model.train()
    optimiser.zero_grad()

    p_data = model(features, edge_index)
    loss = 0

    for i in range(k):

        aug_edge_index = aug_edge_indexes[i]

        shuffle_features = features
        idx = np.random.permutation(nb_nodes)
        shuffle_features = features[:, idx, :]


        p_neigh = model(shuffle_features, aug_edge_index)

        c_theta_j1 = p_neigh/p_data
        c_theta_j2 = p_data/p_neigh

        j1 = (c_theta_j1**2 + 2 * c_theta_j1).mean()
        j2 = (2 * c_theta_j2).mean()



        neigh_loss = j1 - j2
        neigh_loss = neigh_loss.mean()
        loss += neigh_loss

    loss = loss / k

    losses.append(loss)



    logits = p_data.squeeze().detach().cpu()
    auc_score = eval_roc_auc(y.numpy(), logits.numpy()) * 100

    print("Epoch: ", epoch, " Loss: ", loss.item(), " AUC Score: ", auc_score)
    mx_auc = max(mx_auc, auc_score)

    loss.backward()
    optimiser.step()

en = time.time()


print("Maximum AUC: ", mx_auc)
print("Required Time: ", en-st)

please type down your desired dataset and press enter: 
 Choose one among this list: 
 weibo, disney, reddit, books
disney
your dataset disney is being processed.
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Namespace(f='/root/.local/share/jupyter/runtime/kernel-8f4075d1-faa4-446c-a74a-7deb32fe88d4.json', dataset='disney', perturb_percent=0.05, seed=42, nb_epochs=200, hidden_dim=16, lr=0.01, l2_coef=0, gpu=0, save_name='try.pkl', drop_edge=True, add_edge=False, self_loop=True, preprocess_feat=True, use_avg_deg=False, num_neigh=1)
------------------------------------------------------------------------------------------------------------

In [15]:
args = reddit_parser.parse_args()
dataset_str = args.dataset
data = load_data(dataset_str)
# data.x.shape
# print(data)

df = pd.DataFrame(data)

df.describe()


,0,1
count,3,3
unique,3,3
top,x,"[[tensor(0.0168), tensor(0.0151), tensor(0.015..."
freq,1,1
